In [5]:
import os
import pandas as pd
import numpy as np
from pprint import pprint


wp = os.getcwd()

def generateArraies(xps, yps, zps, xrc, yrc, zrc):
    psum = xps * yps * zps
    coordinate_set = [[xi,yi,zi,0 if (xi!=0 and xi!=xps-1 and yi!=0 and yi!=yps-1 and zi!=0 and zi!=zps-1) else 1] for zi in range(zps) for yi in range(yps) for xi in range(xps)]
    real_coordinate_set = [[xi, yi, zi] for zi in zrc for yi in yrc for xi in xrc]
    number_set = [_ for _ in range(psum)]
    point_set = {ns:coordinate_set[i] for i,ns in enumerate(number_set)}
    coordinate_set = {ns:real_coordinate_set[i] for i,ns in enumerate(number_set)}
    point_set = pd.DataFrame(point_set).T
    point_set.columns = ['x','y','z','surface']
    coordinate_set = pd.DataFrame(coordinate_set).T
    coordinate_set.columns = ['x','y','z']
    return point_set, xps-1, yps-1, zps-1, coordinate_set

def generatePointPare(idArr, hnum, vnum, tp):
    hnum, vnum = hnum+1, vnum+1
    tp = [1, -1] if tp==1 else [-1, 1]
    n_idArr = np.reshape(idArr, (vnum, hnum))
    pdict = {}
    for vi in range(vnum):
        pdict[vi] = {1: [[n_idArr[vi][hi], n_idArr[vi][hi+1]] for hi in range(hnum-1)],
                    -1: [[n_idArr[vi][hi+1], n_idArr[vi][hi]] for hi in range(hnum-1)]}
    
    for pi in range(vnum-1):
        arr_a, arr_b = pdict[pi][tp[0]], pdict[pi+1][tp[1]]
        try:
            res += [ai+arr_b[i] for i,ai in enumerate(arr_a)]
        except:
            res = [ai+arr_b[i] for i,ai in enumerate(arr_a)]
    
    return res

def generateBlock(df, xls, yls, zls):
    for zi in range(zls):
        df_xy_0 = df[df['z']==zi]
        df_xy_1 = df[df['z']==zi+1]
        XY_0 = generatePointPare(df_xy_0.index.values,xls,yls,1)
        XY_1 = generatePointPare(df_xy_1.index.values,xls,yls,1)
        try:
            blocks += [xy0+XY_1[i] for i,xy0 in enumerate(XY_0)]
        except:
            blocks = [xy0+XY_1[i] for i,xy0 in enumerate(XY_0)]
    
    return pd.DataFrame(blocks)

def generatePatch(df, xls, yls, zls, wd="X+"):
    df_xy_0 = df[df['z']==0]
    df_xy_1 = df[df['z']==zls]
    df_yz_0 = df[df['x']==0]
    df_yz_1 = df[df['x']==xls]
    df_xz_0 = df[df['y']==0]
    df_xz_1 = df[df['y']==yls]
    XY_0 = pd.DataFrame(generatePointPare(df_xy_0.index.values,xls,yls,-1))
    XY_1 = pd.DataFrame(generatePointPare(df_xy_1.index.values,xls,yls,1))
    YZ_0 = pd.DataFrame(generatePointPare(df_yz_0.index.values,yls,zls,-1))
    YZ_1 = pd.DataFrame(generatePointPare(df_yz_1.index.values,yls,zls,1))
    XZ_0 = pd.DataFrame(generatePointPare(df_xz_0.index.values,xls,zls,1))
    XZ_1 = pd.DataFrame(generatePointPare(df_xz_0.index.values,xls,zls,-1))
    if wd == "X+":
        INLET, OUTLET, LEFTWALL, RIGHTWALL, TOP, BOTTOM = YZ_0, YZ_1, XZ_1, XZ_0, XY_1, XY_0
    if wd == "Y+":
        INLET, OUTLET, LEFTWALL, RIGHTWALL, TOP, BOTTOM = XZ_0, XZ_1, YZ_0, YZ_1, XY_1, XY_0
    patches = {
        "INLET"    : INLET,
        "OUTLET"   : OUTLET,
        "LEFTWALL" : LEFTWALL,
        "RIGHTWALL": RIGHTWALL,
        "TOP"      : TOP,
        "BOTTOM"   : BOTTOM
    }
    
    return patches

def generateOutput(df, quote=True):
    rownum = df.shape[0]
    leftside = pd.DataFrame(['(' for _ in range(rownum)])
    if quote:
        rightside = pd.DataFrame(['),' if _<rownum-1 else ')' for _ in range(rownum)])
    else:
        rightside = pd.DataFrame([')' for _ in range(rownum)])
    
    return pd.concat([leftside,df.reset_index(drop=True),rightside],axis=1)



<b>set your X, Y and Z coordinates into list </b>***xrc***, ***yrc*** and ***zrc***<b>, and then run all cells.</b>

In [6]:
xrc = [0,22.18,45.533,60.04]
yrc = [10.28,35.5014,65.933,121.7]
zrc = [100,120,500]

df, xls, yls, zls, realc = generateArraies(len(xrc), len(yrc), len(zrc), xrc, yrc, zrc)
block = generateBlock(df, xls, yls, zls)
patch = generatePatch(df, xls, yls, zls, "Y+")

out_block = generateOutput(block, False)
out_block.to_csv(os.path.join(wp,"BLOCK.TXT"), header=False, index=False, sep=' ')

for k in patch.keys():
    out_patch = generateOutput(patch[k], False)
    out_patch.to_csv(os.path.join(wp,"%s.TXT" % (k)), header=False, index=False, sep=' ')
    
out_coordinate = generateOutput(realc, False)
out_coordinate.to_csv(os.path.join(wp,"COORDINATE.TXT"), header=False, index=False, sep=' ')
